In [1]:
import requests
import grequests
import pandas as pd
import numpy as np
import time

c:\users\bryce\appdata\local\programs\python\python37\lib\site-packages\grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (c:\\users\\bryce\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)
c:\users\bryce\appdata\local\programs\python\python37\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


The following code is how multiple dataframes should be combined `df = pd.concat([df1, df2], ignore_index=True, sort=False)` where the first parameter is a list of all the partial dataframes

Now to test stuff

In [2]:
key = open('key.txt').read()

In [3]:
URL = "http://api.steampowered.com/ISteamApps/GetAppList/v2"
PARAMS = {'key': key}
re = requests.get(url = URL, params = PARAMS)
apps = re.json()['applist']['apps']

In [4]:
apps1 = apps[0:50]

In [21]:
categories = [
    ['name'],
    ['type'],
    ['steam_appid'],
    ['developers'],
    ['publishers'],
    ['is_free'],
    ['price_overview', 'initial'],
    ['achievements', 'total'],
    ['release_date', 'date'],
    ['metacritic', 'score']
]

filters = ''
for cat in categories:
    filters += cat[0] + ','
filters += 'basic,genres,categories'

In [22]:
def fetch(game_info, categories):
    info = {}
    for category in categories:
        cur_info = game_info
        failed = False
        
        for subpart in category:
            if not failed and subpart in cur_info.keys():
                cur_info = cur_info[subpart]
            else:
                failed = True
        
        if not failed:
            info[' '.join(category)] = cur_info
        else:
            info[' '.join(category)] = np.NaN
    
    if 'genres' in game_info.keys():
        for genre in game_info['genres']:
            info['Genre: ' + genre['description']] = True
    if 'categories' in game_info.keys():
        for genre in game_info['categories']:
            info['Category: ' + genre['description']] = True
    
    
    return info

In [23]:
start = time.time()

all_info = []

app_ids = [str(app['appid']) for app in apps1]
detail_urls = [f"http://store.steampowered.com/api/appdetails/?appids={app_id}" for app_id in app_ids]
detail_params = {'filters': filters}

rs = (grequests.get(url = u, params = detail_params) for u in detail_urls)
results = grequests.map(rs)

i = 0
review_urls = []
review_params = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
while i < len(results):
    app_id = app_ids[i]
    r = results[i]
    if r.json()[app_id]['success']:
        game_info = r.json()[app_id]['data']
        info = fetch(game_info, categories)
        all_info.append(info)
        review_urls.append(f"http://store.steampowered.com/appreviews/{app_id}?json=1")
    i += 1

rs = (grequests.get(url = u, params = review_params) for u in review_urls)
results = grequests.map(rs)
i = 0
while i < len(results):
    r = results[i]
    summary = r.json()
    if 'query_summary' in summary.keys():
        q_summary = summary['query_summary']

        if 'total_positive' in q_summary:
            all_info[i]['total_positive'] = q_summary['total_positive']
        else:
            all_info[i]['total_positive'] = np.NaN

        if 'total_reviews' in q_summary:
            all_info[i]['total_reviews'] = q_summary['total_reviews']
        else:
            all_info[i]['total_reviews'] = np.NaN
    
    i += 1



end = time.time()
print(end - start)

14.848646879196167


In [24]:
df1 = pd.DataFrame.from_dict(all_info)
df1.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,...,Category: Cross-Platform Multiplayer,Category: PvP,Category: Online PvP,Category: Online Co-op,Genre: Early Access,Category: VR Support,Genre: Free to Play,Category: In-App Purchases,Genre: Sports,Category: Game demo
0,Orion: A Sci-Fi Visual Novel,game,381010,[Cylight Studios],[Cylight Studios],False,1299.0,NaN,"Sep 28, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky Rogue,game,381020,[Fractal Phase],[Fractal Phase],False,1999.0,28.0,"Aug 24, 2017",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Ritual on Weylyn Island,game,381050,[zemaGamez],[zemaGamez],False,1199.0,5.0,"Dec 4, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ted by Dawn,game,381090,[Proteus Pixel],[Proteus Pixel],False,NaN,50.0,"Jun 29, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Men of Yoshiwara: Kikuya,game,381100,[Dogenzaka Lab],[Dogenzaka Lab],False,2999.0,17.0,"Oct 14, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
start = time.time()

all_info2 = []

for app in apps1:
    app_id = app['appid']
    # make main api query
    URL = "http://store.steampowered.com/api/appdetails"
    PARAMS = {'appids': app_id, 'filters': filters}
    re = requests.get(url = URL, params = PARAMS)
    if re.json()[str(app_id)]['success']:
        game_info = re.json()[str(app_id)]['data']
        info = fetch(game_info, categories)
        # make review api query
        URL = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
        PARAMS = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
        re = requests.get(url = URL, params = PARAMS)
        summary = re.json()
        if 'query_summary' in summary.keys():
            q_summary = summary['query_summary']

            if 'total_positive' in q_summary:
                info['total_positive'] = q_summary['total_positive']
            else:
                info['total_positive'] = np.NaN

            if 'total_reviews' in q_summary:
                info['total_reviews'] = q_summary['total_reviews']
            else:
                info['total_reviews'] = np.NaN


        all_info2.append(info)
        
end = time.time()
print(end - start)

32.92436194419861


In [26]:
df2 = pd.DataFrame.from_dict(all_info2)
df2.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,...,Category: Cross-Platform Multiplayer,Category: PvP,Category: Online PvP,Category: Online Co-op,Genre: Early Access,Category: VR Support,Genre: Free to Play,Category: In-App Purchases,Genre: Sports,Category: Game demo
0,Orion: A Sci-Fi Visual Novel,game,381010,[Cylight Studios],[Cylight Studios],False,1299.0,NaN,"Sep 28, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky Rogue,game,381020,[Fractal Phase],[Fractal Phase],False,1999.0,28.0,"Aug 24, 2017",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Ritual on Weylyn Island,game,381050,[zemaGamez],[zemaGamez],False,1199.0,5.0,"Dec 4, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ted by Dawn,game,381090,[Proteus Pixel],[Proteus Pixel],False,NaN,50.0,"Jun 29, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Men of Yoshiwara: Kikuya,game,381100,[Dogenzaka Lab],[Dogenzaka Lab],False,2999.0,17.0,"Oct 14, 2015",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df2.equals(df1)

True